In [1]:
from pathlib import Path
import json
import pandas as pd
import numpy as np

In [32]:
policy_dict = {'Technical' : 6, 'Supplies' : 4, 'Credit' : 1, 'Fine' : 3, 'Guarantee' : 1, 'Tax' : 5,
 'Direct' : 2, 'General' : 0, 'Unknown' : 0, 'Other' : 0, 'Nan' : 0 }

incentive_dict = {'Incentive' : 1, 'Disincentive' : 1, 'Unknown' : 0, 'Nan' : 0}

In [116]:
rater = "Rater2"
# paths = Path("C:/Users/user/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/Final_input_data/Rater2")
# paths = Path("C:/Users/jordi/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/Final_input_data/Rater2")
paths = Path("C:/Users/jordi/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/Final_input_data/" + rater).glob('**/*.xlsx')

def clean_df(df, reference_column_to_drop_na):
    df.dropna(subset = [reference_column_to_drop_na], inplace = True)
    df.dropna(axis=1, how='all', inplace = True)
    df.replace(np.nan, 'Nan', regex=True, inplace = True)
    return df

def process_new_labels(filename, reference_column_to_drop_na, policy):
    df_temp = pd.concat(pd.read_excel(filename, engine='openpyxl', sheet_name = None), ignore_index = True)
    df_temp = clean_df(df_temp, reference_column_to_drop_na)
    df_temp.insert(0, "Document", df_temp.apply(lambda row: row.Sentence_Id.split("_")[0], axis = 1))
    df_temp.loc[df_temp['Is_policy'] == 0, 'Is_policy'] = "Nan"
    df_temp.loc[df_temp['Is_policy'] == 1, 'Is_policy'] = policy
    if "Other_instrument"in df_temp.columns:
        df_temp['Is_policy'] = np.where(df_temp['Is_policy'] == "Nan", df_temp['Other_instrument'], df_temp['Is_policy'])
    df_temp.loc[df_temp['Is_incentive'] == 0, 'Is_incentive'] = "Unknown"
    df_temp.loc[df_temp['Is_incentive'] == 1, 'Is_incentive'] = "Incentive"
    return df_temp


results_list = []
for path in paths:
    # because path is object not string
    path_in_str = str(path)
    print(path_in_str)
    flag = False
    for policy in policy_dict:
        if policy in path_in_str:
            policy_instrument = policy
            flag = True
    if flag:
        df = process_new_labels(path_in_str, "Is_incentive", policy_instrument)
        results_list.append(df[["Document", "Sentence_Id", "Sentence", "Is_policy", "Is_incentive"]].values.tolist())
    
# data = df[["Document", "Sentence", "Primary_Instrument", "Category"]].to_numpy()
# filename = "WRI_Policy_Tags.xlsx"
# file = paths / filename
# df = pd.concat(pd.read_excel(file, engine='openpyxl', sheet_name = None), ignore_index = True)


C:\Users\jordi\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater2\Copy of Unique_sentence_IDs_Direct_payment.xlsx
C:\Users\jordi\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater2\Copy of Unique_sentence_IDs_Fine.xlsx
C:\Users\jordi\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater2\Copy of Unique_sentence_IDs_Supplies.xlsx
C:\Users\jordi\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater2\Copy of Unique_sentence_IDs_Tax_deduction.xlsx
C:\Users\jordi\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater2\Copy of Unique_sentence_IDs_Technical_assistance.xlsx
C:\Users\jordi\Google Drive\Els_meus_documents\projectes\CompetitiveIntelligence\WRI\Notebooks\Data\Final_input_data\Rater2\WRI_Policy_Tags.xlsx


In [117]:
def label_cleaning(dictionaryionary, label):
    flag = 1
    
    for key in dictionaryionary:
#         print(key, "----", label)
        if key in label:
            return dictionaryionary[key]
            flag = 0
            break
    if flag == 1:
#         print(label)
        return label

dictionary = {}

i = 0
for item1 in results_list:
    for item in item1:
        if item[0] in dictionary:
            dictionary[item[0]]["Seccion unica"]["sentences"][item[1]] = {"text" : item[2], "labels" : label_cleaning(policy_dict, item [3]), "incentive": label_cleaning(incentive_dict, item [4])}
        else:
            dictionary[item[0]] = {"Seccion unica" : {"tags" : [], "sentences" : {}}}
            dictionary[item[0]]["Seccion unica"]["sentences"][item[1]] = {"text" : item[2], "labels" : label_cleaning(policy_dict, item [3]), "incentive": label_cleaning(incentive_dict, item [4])}


In [118]:
path = Path("C:/Users/jordi/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/Final_input_data/Rater3")
filename = "WRI_Policy_Tags.xlsx"
file = path / filename
df = pd.concat(pd.read_excel(file, engine='openpyxl', sheet_name = None), ignore_index = True)

df = clean_df(df, "Document")

In [119]:
print(df.shape)
df.head()

(687, 8)


,Document,Sentence,Primary_Instrument,Other_Instrument,Land_Use_Type,Category,Unique_Policy,Key_words
0,2019 ACUERDO por el que se emiten los Lineamie...,Generar empleo y garantizara la población camp...,Technical assistance,"Supplies, Credit, Direct payment","Forest, Agriculture (Crop)",Incentive,1 (Sembrando Vida),"insumo, crédito, capacitación, asistencia técnica"
1,2019 ACUERDO por el que se emiten los Lineamie...,El Programa incentivará a los sujetos agrarios...,General incentive,Nan,"Forest, Agriculture (Crop)",Incentive,1 (Sembrando Vida),incentivar
2,2019 ACUERDO por el que se emiten los Lineamie...,Los sujetos agrarios beneficiados por el progr...,Supplies,Nan,"Forest, Agriculture (Crop)",Incentive,1 (Sembrando Vida),"apoyo económico, apoyos en especie, insumos, h..."
3,2019 ACUERDO por el que se emiten los Lineamie...,"Adicionalmente, contarán con acompañamiento té...",Technical assistance,Nan,"Forest, Agriculture (Crop)",Incentive,1 (Sembrando Vida),Nan
4,2019 ACUERDO por el que se emiten los Lineamie...,"El sujeto de derecho, recibirá un apoyo económ...",Direct payment,Nan,"Forest, Agriculture (Crop)",Incentive,1 (Sembrando Vida),pesos


In [120]:
i = 0
for item in data:
    if item[0] in dictionary:
        i += 1
        dictionary[item[0]]["Seccion unica"]["sentences"][str(i)] = {"text" : item[1], "labels" : label_cleaning(policy_dict, item [2]), "incentive": label_cleaning(incentive_dict, item [3])}
    else:
        i = 0
        dictionary[item[0]] = {"Seccion unica" : {"tags" : [], "sentences" : {}}}
        dictionary[item[0]]["Seccion unica"]["sentences"][str(i)] = {"text" : item[1], "labels" : label_cleaning(policy_dict, item [2]), "incentive": label_cleaning(incentive_dict, item [3])}

In [121]:
print(len(dictionary))
dictionary

395


{'1739c19': {'Seccion unica': {'tags': [],
   'sentences': {'1739c19_8': {'text': 'Dichos recursos están disponibles para los Fondos, con arreglo a las siguientes normas del Programa de Financiamiento a Fondos de Inversión de Capital de Riesgo para el Fomento Forestal y Maderero - Fondo Forestal y Maderero, en adelante el "Programa"',
     'labels': 0,
     'incentive': 0},
    '1739c19_256': {'text': 'Búsqueda, selección, administración, gestión y financiamiento de empresas relacionadas al sector forestal;',
     'labels': 0,
     'incentive': 1},
    '1739c19_45': {'text': 'Las empresas del numeral anterior deberán financiar directamente la ejecución de planes de forestación en terrenos de aptitud preferentemente forestal de pequeños y medianos propietarios que pertenezcan a alguno de los siguientes grupos:',
     'labels': 2,
     'incentive': 1},
    '1739c19_265': {'text': 'Estrategias y proceso de desinversión, así como esquemas de beneficios, respecto al reparto de las utilidade

In [122]:
sentence_dict = {}
i = 0
for item, value in dictionary.items():
    for key, sentences in value["Seccion unica"]["sentences"].items():
        i += 1
        sentence_dict[sentences["text"]] = [sentences["labels"], sentences["incentive"]]
print(i)
        

1408


In [123]:
sentence_dict

{'Dichos recursos están disponibles para los Fondos, con arreglo a las siguientes normas del Programa de Financiamiento a Fondos de Inversión de Capital de Riesgo para el Fomento Forestal y Maderero - Fondo Forestal y Maderero, en adelante el "Programa"': [0,
  0],
 'Búsqueda, selección, administración, gestión y financiamiento de empresas relacionadas al sector forestal;': [0,
  1],
 'Las empresas del numeral anterior deberán financiar directamente la ejecución de planes de forestación en terrenos de aptitud preferentemente forestal de pequeños y medianos propietarios que pertenezcan a alguno de los siguientes grupos:': [2,
  1],
 'Estrategias y proceso de desinversión, así como esquemas de beneficios, respecto al reparto de las utilidades del proyecto forestal': [0,
  0],
 'Para ello, se busca que los Fondos de Inversión utilicen los recursos disponibles para invertir en pequeñas o medianas empresas dedicadas al desarrollo de proyectos de forestación en el territorio nacional, en pre

In [124]:
path = Path("C:/Users/jordi/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/Final_input_data/Rater2")
filename = "Rater_2_labeled.json"
file = path / filename
with open(file, 'w') as fp:
    json.dump(dictionary, fp)

In [125]:
path = Path("C:/Users/jordi/Google Drive/Els_meus_documents/projectes/CompetitiveIntelligence/WRI/Notebooks/Data/Final_input_data/Rater2")
filename = "Rater_2_labeled_sentences.json"
file = path / filename
with open(file, 'w') as fp:
    json.dump(sentence_dict, fp)